In [3]:
#imports
import tarfile
from extract import load_alexnet, create_image_dataloader, extract_activations, save_activations, ActivationExtractor, HDF5ActivationsDataset, create_activations_dataloader_from_h5
from csae import CSAE, test_csae
import matplotlib.pyplot as plt
from hook import register_hooks, get_activation
import yaml
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:

dataloader = create_image_dataloader(batch_size=1)
model = load_alexnet()

#print(activations.keys())
print(model)






AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [15]:

config_path = 'config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print(config['training'].keys())

out_dir = '.../output'

with open(out_dir + "/config.yaml", "w") as yaml_file:
    yaml.dump(config.to_dict(), yaml_file, default_flow_style=False)


dict_keys(['curriculum', 'save_every_steps', 'keep_every_steps', 'test_run'])


In [7]:
dl = create_image_dataloader(batch_size=1)
print(len(dl))

for i, (img, label) in enumerate(dl):
    print(img.shape)
    print(label)
    print(model.forward(img).sum())
    if i == 0:
        break

1281167
torch.Size([1, 3, 224, 224])
tensor([487])
tensor(2.2009, grad_fn=<SumBackward0>)


In [5]:


encoder = nn.Conv2d(2, 3, 1, 1, 0, bias=False)
inp = torch.randn(1, 2, 3, 3)
print(inp)
print(encoder(inp))
print(F.relu(encoder(inp)))


tensor([[[[ 1.9525, -0.5966,  3.1196],
          [ 0.8031, -0.3064,  0.3612],
          [ 0.1897,  2.0234,  0.0905]],

         [[-0.2618,  0.4306, -0.9622],
          [ 0.7026, -0.7585, -0.0704],
          [-0.1329,  1.0488, -1.9861]]]])
tensor([[[[ 0.4619,  0.0517,  0.4389],
          [ 0.6356, -0.5122,  0.0734],
          [-0.0142,  1.2046, -1.0642]],

         [[ 0.9807, -0.2464,  1.4843],
          [ 0.5266, -0.2755,  0.1781],
          [ 0.0789,  1.2171, -0.2547]],

         [[-0.8535,  0.3974, -1.5755],
          [-0.0353, -0.1776, -0.1665],
          [-0.1248, -0.3701, -0.8087]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[0.4619, 0.0517, 0.4389],
          [0.6356, 0.0000, 0.0734],
          [0.0000, 1.2046, 0.0000]],

         [[0.9807, 0.0000, 1.4843],
          [0.5266, 0.0000, 0.1781],
          [0.0789, 1.2171, 0.0000]],

         [[0.0000, 0.3974, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]]]], grad_fn=<ReluBackward0>)


In [3]:
print(activations['features.2'].shape)

torch.Size([128, 64, 27, 27])


In [4]:
image, label = dataloader.dataset[0]

model_1 = load_alexnet()
model_2 = load_alexnet()

target_layers = ['features.2', 'features.5', 'features.7', 'features.9', 'features.12', 'avgpool']

activations_1 = {layer: [] for layer in target_layers}




activation_extractor = ActivationExtractor(model_2, target_layers)

register_hooks(model_1, activations_1)

e = model_1.features(image.unsqueeze(0))
f = model_1.avgpool(e)
model_2(image.unsqueeze(0))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([[ 1.1806e+01, -1.2378e+00, -1.2017e+00, -2.4257e-01,  1.3279e+00,
          6.4901e-01, -1.9912e+00, -7.4751e-01, -2.8881e+00, -2.8436e+00,
         -1.5033e+00, -2.0983e+00, -1.3609e+00, -3.9040e+00, -5.3155e-01,
         -2.2305e+00, -4.5163e+00, -1.9538e+00, -3.0487e+00, -1.9249e+00,
         -1.6554e+00,  1.5341e+00, -8.3093e-01, -9.1131e-01,  1.6950e-01,
         -2.3193e+00, -1.3676e+00, -4.4765e-01, -1.1091e-01, -1.0086e+00,
          1.9565e-01, -5.2310e-01,  1.0101e-01,  8.0518e-01,  2.2064e+00,
          2.6919e-01,  2.0391e+00,  8.9842e-01, -1.8357e+00, -1.3219e+00,
         -2.7920e+00, -2.8715e-01, -1.8757e+00,  3.4403e-02,  8.6331e-01,
         -7.8539e-01, -1.4949e+00, -9.5772e-01, -2.7194e+00,  8.4934e-01,
          1.3721e+00, -7.6536e-01,  2.9027e+00,  8.1893e-01,  8.1744e-01,
         -8.3440e-02, -2.1824e+00, -6.7329e-01, -1.4236e-01,  2.8496e+00,
          5.8552e-01,  6.4762e-01,  3.7189e+00,  1.6072e+00,  7.6439e-01,
          1.5029e+00, -3.5418e-01,  2.

In [6]:
print(activations_1.keys())
print(activation_extractor.activations.keys())

for keys in activations_1.keys():
    #print(activations_1[keys] == activation_extractor.activations[keys][0])
    print(keys)
    print(activation_extractor.activations[keys][0].shape)

dict_keys(['features.2', 'features.5', 'features.7', 'features.9', 'features.12', 'avgpool'])
dict_keys(['features.2', 'features.5', 'features.7', 'features.9', 'features.12', 'avgpool'])
features.2
torch.Size([1, 64, 27, 27])
features.5
torch.Size([1, 192, 13, 13])
features.7
torch.Size([1, 384, 13, 13])
features.9
torch.Size([1, 256, 13, 13])
features.12
torch.Size([1, 256, 6, 6])
avgpool
torch.Size([1, 256, 6, 6])


In [ ]:
print(activations_1.keys())
print()

dict_keys(['features.2', 'features.5', 'features.7', 'features.9', 'features.12', 'avgpool'])


In [ ]:
#need to design the c-sae model, create training loop, and the loss function, then that should be it for the c-sae project for now